# Exploration du modèle Chronos-2

Ce notebook permet d'explorer la structure du modèle Chronos-2 depuis Hugging Face.


## 1. Installation et imports


In [ ]:
# Configuration SSL pour macOS
import ssl
import certifi

# Utiliser les certificats de certifi
ssl._create_default_https_context = ssl._create_unverified_context

print("✓ Configuration SSL appliquée")


In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoConfig
from huggingface_hub import list_repo_files, model_info

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")


## 2. À propos de Chronos-2

**Chronos-2** est un modèle de fondation pour la prévision de séries temporelles développé par Amazon. 

Il supporte :
- Prévisions **univariées** (une seule série)
- Prévisions **multivariées** (plusieurs séries simultanément)
- Prévisions **informées par des covariables** (variables externes)

Tout cela dans une architecture unique et unifiée !


In [ ]:
# Utiliser le modèle Chronos-2 officiel
model_name = "amazon/chronos-2"

print(f"Modèle sélectionné: {model_name}")
print("Chronos-2 est la dernière version du modèle Amazon pour la prévision de séries temporelles")


## 3. Informations sur le modèle depuis Hugging Face


In [ ]:
# Récupérer les informations du modèle
info = model_info(model_name)

print(f"Nom du modèle: {info.modelId}")
print(f"Auteur: {info.author}")
print(f"Tags: {info.tags}")
print(f"Pipeline: {info.pipeline_tag}")
print(f"Dernière modification: {info.lastModified}")


In [ ]:
# Lister les fichiers du repository
files = list_repo_files(model_name)
print("\nFichiers dans le repository:")
for file in files:
    print(f"  - {file}")


## 4. Chargement de la configuration du modèle


In [ ]:
# Charger la configuration sans télécharger les poids
config = AutoConfig.from_pretrained(model_name)

print("Configuration du modèle:")
print(f"  - Architecture: {config.model_type}")
print(f"  - Nombre de couches: {config.num_layers}")
print(f"  - Nombre de têtes d'attention: {config.num_heads}")
print(f"  - Dimension du modèle: {config.d_model}")
print(f"  - Dimension FFN: {config.d_ff}")
print(f"  - Taille du vocabulaire: {config.vocab_size}")
print(f"  - Longueur max: {config.n_positions if hasattr(config, 'n_positions') else 'N/A'}")


In [ ]:
# Afficher toute la configuration
print("\nConfiguration complète:")
print(config)


## 5. Téléchargement et chargement du modèle

**Attention**: Le téléchargement peut prendre du temps selon la taille du modèle et votre connexion internet.


In [ ]:
# Charger le modèle Chronos-2 avec la bonne classe
from chronos import Chronos2Pipeline

print("Téléchargement du modèle Chronos-2 en cours...")
print("Note: Le téléchargement peut prendre plusieurs minutes...")

# Utiliser Chronos2Pipeline au lieu de AutoModel pour charger correctement les poids
pipeline = Chronos2Pipeline.from_pretrained(
    model_name, 
    device_map="mps",  # "mps" pour Apple Silicon, "cuda" pour GPU NVIDIA, "cpu" pour CPU
    torch_dtype="auto"
)
print("Modèle Chronos-2 chargé avec succès!")
print(f"Device utilisé: {pipeline.model.device}")


## 6. Exploration de la structure du modèle


In [ ]:
# Afficher l'architecture du modèle
print("Architecture du modèle:")
print(pipeline.model)


In [ ]:
# Compter les paramètres
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

# Accéder au modèle T5 sous-jacent dans le pipeline
total, trainable = count_parameters(pipeline.model)
print(f"\nNombre total de paramètres: {total:,}")
print(f"Paramètres entraînables: {trainable:,}")
print(f"Taille approximative en mémoire: {total * 4 / (1024**2):.2f} MB (float32)")


In [ ]:
# Lister tous les modules du modèle
print("\nModules principaux:")
for name, module in pipeline.model.named_children():
    print(f"  - {name}: {type(module).__name__}")


In [ ]:
# Explorer les couches en détail
print("\nCouches du modèle (premier niveau):")
for name, param in pipeline.model.named_parameters():
    if param.requires_grad:
        print(f"  - {name}: {param.shape}")


## 7. Analyse des composants clés


In [ ]:
# Analyser l'encoder
if hasattr(pipeline.model, 'encoder'):
    print("Structure de l'Encoder:")
    print(pipeline.model.encoder)
    print(f"\nNombre de couches dans l'encoder: {len(list(pipeline.model.encoder.children()))}")


In [ ]:
# Analyser le decoder
if hasattr(pipeline.model, 'decoder'):
    print("Structure du Decoder:")
    print(pipeline.model.decoder)
    print(f"\nNombre de couches dans le decoder: {len(list(pipeline.model.decoder.children()))}")


## 11. Exemple d'utilisation de Chronos-2 pour la prévision

Voici comment utiliser Chronos-2 pour faire des prévisions sur vos propres données.


In [ ]:
import pandas as pd  # requires: pip install 'pandas[pyarrow]'
from chronos import Chronos2Pipeline

pipeline = Chronos2Pipeline.from_pretrained("amazon/chronos-2", device_map="mps")

# Load historical target values and past values of covariates
context_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/train.parquet")

# (Optional) Load future values of covariates
test_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/test.parquet")
future_df = test_df.drop(columns="target")

# Generate predictions with covariates
pred_df = pipeline.predict_df(
    context_df,
    future_df=future_df,
    prediction_length=24,  # Number of steps to forecast
    quantile_levels=[0.1, 0.5, 0.9],  # Quantiles for probabilistic forecast
    id_column="id",  # Column identifying different time series
    timestamp_column="timestamp",  # Column with datetime information
    target="target",  # Column(s) with time series values to predict
)


## 12. Paramètres d'inférence du Chronos2Pipeline

Explorons les paramètres disponibles pour l'inférence avec Chronos-2.


In [ ]:
# Examiner la signature de la méthode predict_df
import inspect

print("=== Signature de predict_df ===\n")
print(inspect.signature(pipeline.predict_df))
print("\n" + "="*60 + "\n")

# Afficher la documentation complète
print("=== Documentation de predict_df ===\n")
print(pipeline.predict_df.__doc__)


In [ ]:
# Examiner aussi la méthode predict (pour les arrays numpy/tensors)
print("=== Signature de predict ===\n")
print(inspect.signature(pipeline.predict))
print("\n" + "="*60 + "\n")

print("=== Documentation de predict ===\n")
print(pipeline.predict.__doc__)


## 13. Exemple d'inférence avec paramètres personnalisés

Voici comment utiliser les différents paramètres d'inférence disponibles dans Chronos-2.


In [ ]:
# Créer des données de test simples
import pandas as pd
import numpy as np

# Créer une série temporelle simple pour tester
dates = pd.date_range(start='2024-01-01', periods=100, freq='D')
values = np.sin(np.linspace(0, 4*np.pi, 100)) + np.random.normal(0, 0.1, 100)

# Créer un DataFrame de contexte
simple_context_df = pd.DataFrame({
    'id': ['series_1'] * 100,
    'timestamp': dates,
    'target': values
})

print("Données de test créées:")
print(simple_context_df.head())
print(f"\nTaille: {len(simple_context_df)} observations")


In [ ]:
# Exemple 1: Inférence avec paramètres par défaut
print("=== Inférence avec paramètres par défaut ===\n")

pred_default = pipeline.predict_df(
    simple_context_df,
    prediction_length=10,  # Nombre de pas de temps à prédire
    id_column="id",
    timestamp_column="timestamp",
    target="target"
)

print(pred_default.head(15))
print(f"\nShape: {pred_default.shape}")


In [ ]:
# Exemple 2: Inférence avec quantiles spécifiques
print("=== Inférence avec quantiles personnalisés ===\n")

pred_quantiles = pipeline.predict_df(
    simple_context_df,
    prediction_length=10,
    quantile_levels=[0.05, 0.25, 0.5, 0.75, 0.95],  # Quantiles pour intervalles de confiance
    id_column="id",
    timestamp_column="timestamp",
    target="target"
)

print(pred_quantiles.head(15))
print(f"\nColonnes disponibles: {pred_quantiles.columns.tolist()}")


In [ ]:
# Visualiser les prédictions avec intervalles de confiance
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 6))

# Afficher les données historiques
ax.plot(simple_context_df['timestamp'], simple_context_df['target'], 
        label='Données historiques', linewidth=2, color='blue')

# Extraire les prédictions
pred_filtered = pred_quantiles[pred_quantiles['id'] == 'series_1']

# Afficher la médiane (quantile 0.5)
if '0.5' in pred_filtered.columns:
    ax.plot(pred_filtered['timestamp'], pred_filtered['0.5'], 
            label='Prédiction (médiane)', linewidth=2, color='red', linestyle='--')

# Afficher les intervalles de confiance
if '0.05' in pred_filtered.columns and '0.95' in pred_filtered.columns:
    ax.fill_between(pred_filtered['timestamp'], 
                     pred_filtered['0.05'], 
                     pred_filtered['0.95'],
                     alpha=0.2, color='red', label='IC 90%')

if '0.25' in pred_filtered.columns and '0.75' in pred_filtered.columns:
    ax.fill_between(pred_filtered['timestamp'], 
                     pred_filtered['0.25'], 
                     pred_filtered['0.75'],
                     alpha=0.3, color='red', label='IC 50%')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Valeur', fontsize=12)
ax.set_title('Prévisions Chronos-2 avec intervalles de confiance', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### Paramètres avancés d'inférence

Les paramètres principaux disponibles pour l'inférence avec Chronos-2 incluent généralement :

- **`prediction_length`**: Nombre de pas de temps à prédire
- **`quantile_levels`**: Liste des quantiles pour les prévisions probabilistes (ex: [0.1, 0.5, 0.9])
- **`num_samples`**: Nombre d'échantillons à générer pour les prévisions probabilistes
- **`temperature`**: Contrôle la diversité des prédictions (plus élevé = plus de variance)
- **`top_k`**: Considère uniquement les k tokens les plus probables
- **`top_p`**: Échantillonnage nucleus (considère les tokens dont la prob cumulée < p)
- **`batch_size`**: Taille des lots pour le traitement
- **`limit_prediction_length`**: Limite la longueur maximale des prédictions


In [ ]:
# Exemple 3: Exploration des paramètres disponibles dans le pipeline
print("=== Attributs du pipeline ===\n")

# Afficher les attributs principaux du pipeline
if hasattr(pipeline, '__dict__'):
    print("Attributs disponibles:")
    for key, value in pipeline.__dict__.items():
        if not key.startswith('_'):
            print(f"  - {key}: {type(value).__name__}")

# Vérifier s'il y a une configuration
if hasattr(pipeline, 'model') and hasattr(pipeline.model, 'config'):
    print("\n=== Configuration du modèle ===\n")
    config = pipeline.model.config
    if hasattr(config, 'chronos_config'):
        chronos_cfg = config.chronos_config
        print("Configuration Chronos:")
        for key, value in chronos_cfg.items():
            print(f"  - {key}: {value}")


In [ ]:
# Exemple 4: Prévisions avec num_samples (si disponible)
# Testons différents scénarios pour voir quels paramètres sont acceptés

print("=== Test avec différents paramètres ===\n")

try:
    # Essayer avec num_samples
    pred_samples = pipeline.predict_df(
        simple_context_df,
        prediction_length=10,
        num_samples=100,  # Nombre d'échantillons pour prévisions probabilistes
        id_column="id",
        timestamp_column="timestamp",
        target="target"
    )
    print("✓ num_samples est supporté")
    print(f"  Shape des prédictions: {pred_samples.shape}")
except TypeError as e:
    print(f"✗ num_samples n'est pas supporté ou mal utilisé")
    print(f"  Erreur: {str(e)}")


In [ ]:
# Exemple 5: Tester temperature (si disponible)
try:
    pred_temp = pipeline.predict_df(
        simple_context_df,
        prediction_length=10,
        temperature=0.8,  # Contrôle la diversité des prédictions
        quantile_levels=[0.1, 0.5, 0.9],
        id_column="id",
        timestamp_column="timestamp",
        target="target"
    )
    print("✓ temperature est supporté")
    print(f"  Shape des prédictions: {pred_temp.shape}")
except TypeError as e:
    print(f"✗ temperature n'est pas supporté ou mal utilisé")
    print(f"  Erreur: {str(e)}")


In [ ]:
print("\n=== Utilisation de predict() avec arrays ===\n")

# 1) Série historique univariée
context_tensor = torch.tensor(
    simple_context_df['target'].values,
    dtype=torch.float32
)  # shape: (T,)

# 2) Passage en 3D : (n_series, n_variates, history_length)
#    Ici: (1, 1, T)
context_3d = context_tensor.unsqueeze(0).unsqueeze(0)

# 3) Prédictions
predictions = pipeline.predict(
    inputs=context_3d,    # ou juste pipeline.predict(context_3d, prediction_length=10)
    prediction_length=10
)

print(f"Type des prédictions: {type(predictions)}")
print(f"\nPrédictions:\n{predictions}")

## 14. Résumé des paramètres d'inférence

Voici un résumé des paramètres principaux pour faire de l'inférence avec Chronos-2 :

### Paramètres essentiels pour `predict_df()`:

1. **`context_df`**: DataFrame avec les données historiques (obligatoire)
2. **`prediction_length`**: Nombre de pas de temps à prédire (obligatoire)
3. **`id_column`**: Nom de la colonne identifiant les différentes séries (par défaut: "id")
4. **`timestamp_column`**: Nom de la colonne avec les timestamps (par défaut: "timestamp")
5. **`target`**: Nom de la colonne cible à prédire (obligatoire)
6. **`quantile_levels`**: Liste des quantiles pour prévisions probabilistes (ex: [0.1, 0.5, 0.9])
7. **`future_df`**: DataFrame avec les valeurs futures des covariables (optionnel)
8. **`num_samples`**: Nombre d'échantillons à générer (peut varier selon la version)
9. **`batch_size`**: Taille des lots pour le traitement (optionnel)

### Paramètres essentiels pour `predict()`:

1. **`context`**: Tensor avec les données historiques
2. **`prediction_length`**: Nombre de pas de temps à prédire
3. **`num_samples`**: Nombre d'échantillons à générer
4. **`temperature`**: Température pour l'échantillonnage (peut être disponible)
5. **`top_k`** / **`top_p`**: Paramètres d'échantillonnage (selon la version)


## 15. Exemple complet d'inférence avec tous les paramètres

Voici un exemple complet montrant comment utiliser Chronos-2 avec tous les paramètres principaux.


In [ ]:
# Exemple complet avec configuration personnalisée
print("=== EXEMPLE COMPLET D'INFÉRENCE AVEC CHRONOS-2 ===\n")

# 1. Préparer les données
context_df = simple_context_df.copy()
print(f"1. Données de contexte: {len(context_df)} observations")
print(f"   Période: {context_df['timestamp'].min()} à {context_df['timestamp'].max()}\n")

# 2. Configurer les paramètres d'inférence
prediction_config = {
    'prediction_length': 14,  # Prédire 14 jours
    'quantile_levels': [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99],  # Quantiles multiples
    'id_column': 'id',
    'timestamp_column': 'timestamp',
    'target': 'target'
}

print("2. Configuration des paramètres:")
for param, value in prediction_config.items():
    print(f"   - {param}: {value}")

# 3. Faire les prédictions
print("\n3. Génération des prédictions...")
predictions_df = pipeline.predict_df(
    context_df,
    **prediction_config
)

print(f"   ✓ Prédictions générées: {predictions_df.shape}")
print(f"   ✓ Colonnes: {predictions_df.columns.tolist()}\n")

# 4. Afficher un échantillon des résultats
print("4. Aperçu des prédictions:")
print(predictions_df.head(10))

# 5. Statistiques sur les prédictions
print("\n5. Statistiques des prédictions:")
if '0.5' in predictions_df.columns:
    median_values = predictions_df['0.5']
    print(f"   - Médiane min: {median_values.min():.4f}")
    print(f"   - Médiane max: {median_values.max():.4f}")
    print(f"   - Médiane moyenne: {median_values.mean():.4f}")
    
# 6. Calculer la largeur des intervalles de confiance
if '0.05' in predictions_df.columns and '0.95' in predictions_df.columns:
    ic_90 = predictions_df['0.95'] - predictions_df['0.05']
    print(f"   - Largeur moyenne IC 90%: {ic_90.mean():.4f}")
    
if '0.25' in predictions_df.columns and '0.75' in predictions_df.columns:
    ic_50 = predictions_df['0.75'] - predictions_df['0.25']
    print(f"   - Largeur moyenne IC 50%: {ic_50.mean():.4f}")


In [ ]:
# Visualisation avancée des prédictions avec tous les quantiles
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Préparer les données
pred_plot = predictions_df[predictions_df['id'] == 'series_1'].copy()
hist_plot = context_df[context_df['id'] == 'series_1'].copy()

# --- Graphique 1: Vue d'ensemble avec tous les intervalles de confiance ---
ax1.plot(hist_plot['timestamp'], hist_plot['target'], 
         label='Données historiques', linewidth=2.5, color='#2E86AB', alpha=0.8)

# Médiane
if '0.5' in pred_plot.columns:
    ax1.plot(pred_plot['timestamp'], pred_plot['0.5'], 
             label='Prédiction (médiane)', linewidth=2.5, color='#A23B72', linestyle='--')

# Intervalles de confiance empilés
intervals = [
    ('0.01', '0.99', 0.05, 'IC 98%'),
    ('0.05', '0.95', 0.10, 'IC 90%'),
    ('0.10', '0.90', 0.15, 'IC 80%'),
    ('0.25', '0.75', 0.25, 'IC 50%'),
]

colors = ['#F18F01', '#C73E1D', '#6A994E', '#A23B72']
for i, (q_low, q_high, alpha, label) in enumerate(intervals):
    if q_low in pred_plot.columns and q_high in pred_plot.columns:
        ax1.fill_between(pred_plot['timestamp'], 
                         pred_plot[q_low], 
                         pred_plot[q_high],
                         alpha=alpha, color=colors[i % len(colors)], label=label)

ax1.axvline(x=hist_plot['timestamp'].iloc[-1], color='gray', linestyle=':', 
            linewidth=2, label='Début des prédictions')
ax1.set_xlabel('Date', fontsize=12, fontweight='bold')
ax1.set_ylabel('Valeur', fontsize=12, fontweight='bold')
ax1.set_title('Prévisions Chronos-2 avec intervalles de confiance multiples', 
              fontsize=14, fontweight='bold')
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3, linestyle='--')

# --- Graphique 2: Zoom sur les prédictions uniquement ---
# Médiane
if '0.5' in pred_plot.columns:
    ax2.plot(pred_plot['timestamp'], pred_plot['0.5'], 
             label='Médiane (Q50)', linewidth=2.5, color='#A23B72', marker='o')

# Afficher plusieurs quantiles individuellement
quantiles_to_plot = ['0.05', '0.25', '0.75', '0.95']
quantile_colors = ['#C73E1D', '#F18F01', '#6A994E', '#2E86AB']

for q, color in zip(quantiles_to_plot, quantile_colors):
    if q in pred_plot.columns:
        q_label = f'Q{int(float(q)*100)}'
        ax2.plot(pred_plot['timestamp'], pred_plot[q], 
                 label=q_label, linewidth=1.5, color=color, 
                 marker='s', markersize=4, alpha=0.7)

ax2.set_xlabel('Date', fontsize=12, fontweight='bold')
ax2.set_ylabel('Valeur', fontsize=12, fontweight='bold')
ax2.set_title('Détail des quantiles de prédiction', fontsize=14, fontweight='bold')
ax2.legend(loc='best', fontsize=10, ncol=2)
ax2.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print("\n✓ Visualisation complète des prédictions avec intervalles de confiance")


## 16. Bonnes pratiques pour l'inférence avec Chronos-2

### 🎯 Choix des paramètres

**Quantiles (`quantile_levels`)**:
- Pour des intervalles de confiance standards : `[0.1, 0.5, 0.9]` (IC 80%)
- Pour une analyse détaillée : `[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]`
- Le quantile 0.5 correspond à la médiane (prédiction centrale)

**Longueur de prédiction (`prediction_length`)**:
- Dépend de la fréquence des données et de l'horizon souhaité
- Généralement : 10-20% de la longueur du contexte est raisonnable
- Pour des données journalières : 7-30 jours est courant

**Nombre d'échantillons (`num_samples`)**:
- Plus d'échantillons = prédictions plus lisses mais calcul plus lent
- Valeurs typiques : 20-100 échantillons
- Pour production : 100-1000 échantillons selon la précision requise

### ⚡ Performance

- Utilisez `batch_size` pour traiter plusieurs séries en parallèle
- Pour GPU: augmentez `batch_size` pour maximiser l'utilisation
- Pour CPU: réduisez `batch_size` si vous manquez de mémoire

### 📊 Interprétation des résultats

- Les quantiles extrêmes (0.01, 0.99) capturent les scénarios rares
- L'écart entre quantiles indique l'incertitude du modèle
- Un IC large = haute incertitude, IC étroit = haute confiance


## 17. Utilisation avancée : méthode `predict()` avec tensors

La méthode `predict()` offre un contrôle plus bas niveau et permet de travailler directement avec des tensors PyTorch. Cette approche est utile pour :
- Intégrer Chronos-2 dans un pipeline ML personnalisé
- Avoir un contrôle fin sur les paramètres d'inférence
- Traiter des données déjà sous forme de tensors


In [ ]:
# Examiner en détail la méthode predict et ses paramètres
print("=== Documentation détaillée de la méthode predict() ===\n")

# Afficher la signature
print("Signature:")
print(inspect.signature(pipeline.predict))
print("\n" + "="*70 + "\n")

# Afficher la documentation
if pipeline.predict.__doc__:
    print("Documentation:")
    print(pipeline.predict.__doc__)
else:
    print("Pas de documentation disponible pour predict()")
    
# Explorer les attributs du pipeline
print("\n" + "="*70 + "\n")
print("Type du pipeline:", type(pipeline))
print("Méthodes disponibles:")
for attr in dir(pipeline):
    if not attr.startswith('_') and callable(getattr(pipeline, attr)):
        print(f"  - {attr}")


In [ ]:
# Exemple d'utilisation de predict() avec des tensors
print("=== Exemple avec predict() et tensors ===\n")

# Préparer les données
context_values = simple_context_df['target'].values
context_tensor = torch.tensor(context_values, dtype=torch.float32).unsqueeze(0)  # Shape: (1, seq_len)

print(f"1. Contexte préparé:")
print(f"   - Shape: {context_tensor.shape}")
print(f"   - Type: {context_tensor.dtype}")
print(f"   - Device: {context_tensor.device}\n")

# Faire des prédictions avec paramètres par défaut
try:
    predictions_basic = pipeline.predict(
        context=context_tensor,
        prediction_length=10
    )
    
    print(f"2. Prédictions générées:")
    print(f"   - Type: {type(predictions_basic)}")
    print(f"   - Shape: {predictions_basic.shape}")
    print(f"   - Device: {predictions_basic.device}\n")
    
    print(f"3. Aperçu des prédictions:")
    print(predictions_basic)
    
except Exception as e:
    print(f"Erreur lors de l'inférence: {e}")
    print(f"Type d'erreur: {type(e).__name__}")


In [ ]:
# Essayer avec num_samples pour obtenir plusieurs trajectoires
print("\n=== Test avec num_samples ===\n")

try:
    predictions_samples = pipeline.predict(
        context=context_tensor,
        prediction_length=10,
        num_samples=20  # Générer 20 trajectoires de prédiction
    )
    
    print(f"✓ Prédictions avec échantillons multiples:")
    print(f"   - Shape: {predictions_samples.shape}")
    print(f"   - Expected shape: (num_samples, batch_size, prediction_length)")
    
    # Calculer statistiques
    print(f"\n   Statistiques par pas de temps:")
    mean_pred = predictions_samples.mean(dim=0).squeeze()
    std_pred = predictions_samples.std(dim=0).squeeze()
    
    for i in range(min(5, len(mean_pred))):
        print(f"   - t+{i+1}: {mean_pred[i]:.4f} ± {std_pred[i]:.4f}")
        
except TypeError as e:
    print(f"✗ num_samples non supporté dans cette signature")
    print(f"   Erreur: {e}")
except Exception as e:
    print(f"✗ Erreur: {type(e).__name__}: {e}")


In [ ]:
# Tester d'autres paramètres potentiels
print("\n=== Test de paramètres additionnels ===\n")

# Liste de paramètres à tester
params_to_test = [
    ('temperature', 0.8, "Contrôle la diversité des prédictions"),
    ('top_k', 50, "Limite aux k tokens les plus probables"),
    ('top_p', 0.9, "Échantillonnage nucleus"),
    ('batch_size', 1, "Taille du batch"),
]

for param_name, param_value, description in params_to_test:
    try:
        kwargs = {
            'context': context_tensor,
            'prediction_length': 10,
            param_name: param_value
        }
        
        pred_test = pipeline.predict(**kwargs)
        print(f"✓ {param_name:20s}: Supporté - {description}")
        print(f"  Shape: {pred_test.shape}")
        
    except TypeError as e:
        if "unexpected keyword argument" in str(e):
            print(f"✗ {param_name:20s}: Non supporté dans cette version")
        else:
            print(f"✗ {param_name:20s}: Erreur - {str(e)[:60]}")
    except Exception as e:
        print(f"✗ {param_name:20s}: Erreur - {type(e).__name__}")


## 18. Code template réutilisable pour l'inférence

Voici un template complet que vous pouvez réutiliser pour faire de l'inférence avec Chronos-2 :


In [ ]:
"""
TEMPLATE RÉUTILISABLE POUR L'INFÉRENCE AVEC CHRONOS-2
========================================================

Ce template peut être copié et adapté pour vos propres données.
"""

def make_chronos2_predictions(
    context_df,
    prediction_length=30,
    quantile_levels=None,
    id_column="id",
    timestamp_column="timestamp",
    target_column="target",
    future_df=None,
    pipeline=None
):
    """
    Fait des prédictions avec Chronos-2 sur un DataFrame pandas.
    
    Parameters
    ----------
    context_df : pd.DataFrame
        DataFrame avec les données historiques
    prediction_length : int
        Nombre de pas de temps à prédire
    quantile_levels : list, optional
        Liste des quantiles à calculer (default: [0.1, 0.5, 0.9])
    id_column : str
        Nom de la colonne identifiant les séries
    timestamp_column : str
        Nom de la colonne avec les timestamps
    target_column : str
        Nom de la colonne cible
    future_df : pd.DataFrame, optional
        DataFrame avec les valeurs futures des covariables
    pipeline : Chronos2Pipeline
        Instance du pipeline Chronos-2 (sera créée si None)
    
    Returns
    -------
    pd.DataFrame
        DataFrame avec les prédictions pour chaque quantile
    """
    import pandas as pd
    from chronos import Chronos2Pipeline
    
    # Paramètres par défaut
    if quantile_levels is None:
        quantile_levels = [0.1, 0.5, 0.9]
    
    # Charger le pipeline si nécessaire
    if pipeline is None:
        print("Chargement du pipeline Chronos-2...")
        pipeline = Chronos2Pipeline.from_pretrained(
            "amazon/chronos-2",
            device_map="mps"  # ou "cuda" pour GPU NVIDIA, "cpu" pour CPU
        )
    
    # Configuration de la prédiction
    config = {
        'prediction_length': prediction_length,
        'quantile_levels': quantile_levels,
        'id_column': id_column,
        'timestamp_column': timestamp_column,
        'target': target_column
    }
    
    # Ajouter les covariables si disponibles
    if future_df is not None:
        config['future_df'] = future_df
    
    # Faire les prédictions
    print(f"Génération de {prediction_length} prédictions...")
    predictions_df = pipeline.predict_df(context_df, **config)
    
    print(f"✓ Prédictions générées: {predictions_df.shape}")
    
    return predictions_df

# Exemple d'utilisation du template
print("=== EXEMPLE D'UTILISATION DU TEMPLATE ===\n")

predictions = make_chronos2_predictions(
    context_df=simple_context_df,
    prediction_length=14,
    quantile_levels=[0.05, 0.25, 0.5, 0.75, 0.95],
    id_column="id",
    timestamp_column="timestamp",
    target_column="target",
    pipeline=pipeline  # Réutiliser le pipeline déjà chargé
)

print("\nRésultat:")
print(predictions.head(10))


## 19. Ressources utiles

### Documentation officielle

- **Page du modèle**: https://huggingface.co/amazon/chronos-2
- **GitHub Repository**: https://github.com/amazon-science/chronos-forecasting
- **Paper Chronos-2**: https://arxiv.org/abs/2510.15821

### Paramètres d'inférence principaux

| Paramètre | Description | Valeurs typiques |
|-----------|-------------|------------------|
| `prediction_length` | Horizon de prévision | 7-30 pour données journalières |
| `quantile_levels` | Quantiles pour IC | [0.1, 0.5, 0.9] ou [0.05, 0.25, 0.5, 0.75, 0.95] |
| `num_samples` | Nombre d'échantillons | 20-100 (dev), 100-1000 (prod) |
| `batch_size` | Taille des lots | Selon mémoire disponible |
| `temperature` | Diversité des prédictions | 0.7-1.0 (si disponible) |


---

## ✅ Résumé

Ce notebook vous a montré comment :

1. ✓ Charger et explorer le modèle Chronos-2
2. ✓ Utiliser la méthode `predict_df()` pour faire des prédictions sur des DataFrames
3. ✓ Spécifier les paramètres d'inférence (quantiles, prediction_length, etc.)
4. ✓ Utiliser la méthode `predict()` pour un contrôle bas niveau avec des tensors
5. ✓ Visualiser les prédictions avec des intervalles de confiance
6. ✓ Créer un template réutilisable pour vos propres projets

**Paramètres d'inférence principaux :**
- `prediction_length`: Horizon de prévision
- `quantile_levels`: Quantiles pour prévisions probabilistes
- `num_samples`: Nombre d'échantillons à générer
- `id_column`, `timestamp_column`, `target`: Colonnes du DataFrame
- `future_df`: Covariables futures (optionnel)

Vous êtes maintenant prêt à utiliser Chronos-2 pour vos propres prévisions de séries temporelles ! 🚀


## 20. Utilisation avec votre propre dataset

Cette section montre comment utiliser Chronos-2 avec votre dataset personnalisé.


In [ ]:
# Préparation de votre dataset pour Chronos-2
# 
# Votre dataset doit avoir:
# - Une colonne 'date' (ou similaire) avec les timestamps
# - Des colonnes numériques pour les séries temporelles à prédire
# - Optionnellement: d'autres colonnes comme covariables

print("=== PRÉPARATION DU DATASET ===\n")

# Supposons que vous avez votre DataFrame dans une variable 'df'
# Si ce n'est pas le cas, chargez-le ici:
# df = pd.read_csv('votre_fichier.csv')
# ou
# df = pd.read_excel('votre_fichier.xlsx')

# Afficher les informations du dataset
print("Structure du dataset:")
print(f"  - Nombre de lignes: {len(df)}")
print(f"  - Nombre de colonnes: {len(df.columns)}")
print(f"  - Colonnes: {df.columns.tolist()}\n")

# Afficher les premières lignes
print("Premières lignes:")
print(df.head())
print("\n" + "="*60 + "\n")

# Vérifier les types de données
print("Types de données:")
print(df.dtypes)
print("\n" + "="*60 + "\n")

# Vérifier les valeurs manquantes
print("Valeurs manquantes par colonne:")
missing = df.isnull().sum()
print(missing[missing > 0])


In [ ]:
# Fonction pour préparer le dataset au format Chronos-2
def prepare_data_for_chronos2(
    df,
    date_column='date',
    target_columns=None,
    covariate_columns=None,
    id_value='series_1',
    freq=None,
    fill_method='forward'
):
    """
    Prépare un DataFrame pour l'inférence avec Chronos-2.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame avec les données brutes
    date_column : str
        Nom de la colonne contenant les dates
    target_columns : list or str
        Colonne(s) à prédire. Si None, utilise toutes les colonnes numériques sauf date
    covariate_columns : list
        Colonnes à utiliser comme covariables (optionnel)
    id_value : str
        Identifiant de la série temporelle
    freq : str or None
        Fréquence des données (ex: 'D' pour quotidien, 'M' pour mensuel)
        Si None, sera inférée automatiquement
    fill_method : str
        Méthode pour remplir les valeurs manquantes lors de la création d'une série régulière
        'forward': forward fill, 'backward': backward fill, 'interpolate': interpolation linéaire
    
    Returns
    -------
    pd.DataFrame
        DataFrame formaté pour Chronos-2 avec colonnes: id, timestamp, target, [covariates]
    """
    # Copier le DataFrame
    df_prep = df.copy()
    
    # Convertir la colonne de date en timestamp
    if date_column in df_prep.columns:
        df_prep['timestamp'] = pd.to_datetime(df_prep[date_column])
    else:
        raise ValueError(f"Colonne '{date_column}' introuvable dans le DataFrame")
    
    # Trier par timestamp et supprimer les doublons
    df_prep = df_prep.sort_values('timestamp').drop_duplicates(subset=['timestamp']).reset_index(drop=True)
    
    # Identifier les colonnes numériques (sauf date et timestamp)
    numeric_cols = df_prep.select_dtypes(include=[np.number]).columns.tolist()
    if 'timestamp' in numeric_cols:
        numeric_cols.remove('timestamp')
    
    # Déterminer les colonnes cibles
    if target_columns is None:
        # Par défaut, utiliser toutes les colonnes numériques
        target_columns = numeric_cols
    elif isinstance(target_columns, str):
        target_columns = [target_columns]
    
    # Vérifier que les colonnes cibles existent
    for col in target_columns:
        if col not in df_prep.columns:
            raise ValueError(f"Colonne cible '{col}' introuvable")
    
    # Déterminer les covariables (toutes les autres colonnes numériques)
    if covariate_columns is None:
        covariate_columns = [col for col in numeric_cols if col not in target_columns]
    
    # Inférer ou déterminer la fréquence
    if freq is None:
        # Essayer d'inférer la fréquence à partir des timestamps
        timestamps_sorted = df_prep['timestamp'].sort_values().reset_index(drop=True)
        freq = pd.infer_freq(timestamps_sorted)
        
        if freq is None:
            # Si l'inférence échoue, calculer la fréquence la plus commune
            if len(timestamps_sorted) > 1:
                diffs = timestamps_sorted.diff().dropna()
                most_common_diff = diffs.mode()
                if len(most_common_diff) > 0:
                    # Convertir la différence la plus commune en fréquence
                    diff_value = most_common_diff.iloc[0]
                    if pd.Timedelta(days=1) <= diff_value <= pd.Timedelta(days=1.5):
                        freq = 'D'  # Daily
                    elif pd.Timedelta(days=7) <= diff_value <= pd.Timedelta(days=8):
                        freq = 'W'  # Weekly
                    elif pd.Timedelta(days=28) <= diff_value <= pd.Timedelta(days=32):
                        freq = 'M'  # Monthly
                    elif pd.Timedelta(days=90) <= diff_value <= pd.Timedelta(days=95):
                        freq = 'Q'  # Quarterly
                    else:
                        # Utiliser la différence moyenne comme fréquence personnalisée
                        freq = diff_value
                        print(f"⚠️  Utilisation d'une fréquence personnalisée: {freq}")
                else:
                    raise ValueError("Impossible de déterminer la fréquence des données. Veuillez spécifier 'freq' manuellement.")
            else:
                raise ValueError("Pas assez de données pour inférer la fréquence. Veuillez spécifier 'freq' manuellement.")
    
    print(f"✓ Fréquence utilisée: {freq}")
    
    # Créer une série de timestamps régulière
    min_timestamp = df_prep['timestamp'].min()
    max_timestamp = df_prep['timestamp'].max()
    
    # Créer un index de dates régulières
    if isinstance(freq, str):
        regular_index = pd.date_range(start=min_timestamp, end=max_timestamp, freq=freq)
    else:
        # Si freq est un Timedelta, créer manuellement
        regular_index = pd.date_range(start=min_timestamp, end=max_timestamp, freq=freq)
    
    print(f"✓ Série régulière créée: {len(regular_index)} timestamps de {min_timestamp} à {max_timestamp}")
    
    # Créer le DataFrame au format long pour Chronos-2
    result_dfs = []
    
    for target_col in target_columns:
        # Créer un DataFrame pour cette série avec les timestamps réguliers
        series_df = pd.DataFrame({
            'id': [id_value] * len(regular_index),
            'timestamp': regular_index
        })
        
        # Réindexer les données originales sur les timestamps réguliers
        target_series = df_prep.set_index('timestamp')[target_col]
        target_series = target_series.reindex(regular_index)
        
        # Remplir les valeurs manquantes
        if fill_method == 'forward':
            target_series = target_series.ffill()
        elif fill_method == 'backward':
            target_series = target_series.bfill()
        elif fill_method == 'interpolate':
            target_series = target_series.interpolate(method='linear')
        else:
            # Par défaut, forward fill puis backward fill
            target_series = target_series.ffill().bfill()
        
        series_df['target'] = target_series.values
        
        # Ajouter les covariables si spécifiées
        for cov_col in covariate_columns:
            if cov_col in df_prep.columns:
                cov_series = df_prep.set_index('timestamp')[cov_col]
                cov_series = cov_series.reindex(regular_index)
                
                # Remplir les valeurs manquantes de la même manière
                if fill_method == 'forward':
                    cov_series = cov_series.ffill()
                elif fill_method == 'backward':
                    cov_series = cov_series.bfill()
                elif fill_method == 'interpolate':
                    cov_series = cov_series.interpolate(method='linear')
                else:
                    cov_series = cov_series.ffill().bfill()
                
                series_df[cov_col] = cov_series.values
        
        # Supprimer les lignes où target est toujours NaN (au début si nécessaire)
        series_df = series_df.dropna(subset=['target']).reset_index(drop=True)
        
        result_dfs.append(series_df)
    
    # Si une seule colonne cible, retourner directement
    if len(result_dfs) == 1:
        result_df = result_dfs[0]
    else:
        # Sinon, concaténer avec un id différent pour chaque série
        for i, df_series in enumerate(result_dfs):
            df_series['id'] = f"{id_value}_{target_columns[i]}"
        result_df = pd.concat(result_dfs, ignore_index=True)
    
    # Vérifier que la fréquence est bien régulière
    if len(result_df) > 1:
        inferred_freq = pd.infer_freq(result_df['timestamp'].unique())
        if inferred_freq is None:
            print("⚠️  Attention: La série finale n'a pas une fréquence régulière détectable")
        else:
            print(f"✓ Fréquence finale vérifiée: {inferred_freq}")
    
    return result_df, target_columns, covariate_columns

print("✓ Fonction de préparation créée")


In [ ]:
# Exemple 1: Préparer le dataset pour une seule colonne cible
# Remplacez 'gdp' par la colonne que vous souhaitez prédire

print("=== EXEMPLE 1: PRÉDICTION UNIVARIÉE ===\n")

# Choisir une colonne cible (exemple: 'gdp')
# Vous pouvez changer cette valeur selon vos besoins
target_column = 'gdp'  # Changez selon votre besoin

# Préparer les données
try:
    context_df, targets, covariates = prepare_data_for_chronos2(
        df=df,
        date_column='date',
        target_columns=target_column,  # Prédire une seule colonne
        id_value='ecb_data',
        freq=None  # Sera inféré automatiquement
    )
    
    print(f"✓ Dataset préparé:")
    print(f"  - Colonne cible: {target_column}")
    print(f"  - Covariables: {covariates[:5] if len(covariates) > 5 else covariates}...")
    print(f"  - Nombre de lignes: {len(context_df)}")
    print(f"  - Période: {context_df['timestamp'].min()} à {context_df['timestamp'].max()}\n")
    
    # Afficher un aperçu
    print("Aperçu des données préparées:")
    print(context_df.head(10))
    print(f"\nColonnes: {context_df.columns.tolist()}")
    
except Exception as e:
    print(f"❌ Erreur lors de la préparation: {e}")
    print(f"   Type: {type(e).__name__}")
    import traceback
    traceback.print_exc()


### 📊 Inspection des données d'entrée du modèle

Avant de faire les prédictions, examinons exactement ce qui est donné au modèle Chronos-2.


In [ ]:
# Inspecter les données qui seront passées au modèle
print("=== DONNÉES D'ENTRÉE DU MODÈLE CHRONOS-2 ===\n")

if 'context_df' in globals():
    print("📋 Structure du DataFrame 'context_df' (données historiques):")
    print(f"  - Nombre de lignes: {len(context_df)}")
    print(f"  - Nombre de colonnes: {len(context_df.columns)}")
    print(f"  - Colonnes: {context_df.columns.tolist()}\n")
    
    print("📅 Informations temporelles:")
    print(f"  - Première date: {context_df['timestamp'].min()}")
    print(f"  - Dernière date: {context_df['timestamp'].max()}")
    print(f"  - Période couverte: {(context_df['timestamp'].max() - context_df['timestamp'].min()).days} jours")
    
    # Vérifier la fréquence
    freq = pd.infer_freq(context_df['timestamp'].sort_values())
    if freq:
        print(f"  - Fréquence détectée: {freq}")
    else:
        print(f"  - ⚠️  Fréquence non détectée automatiquement")
    print()
    
    print("🎯 Colonne cible (target):")
    target_col = 'target'
    if target_col in context_df.columns:
        print(f"  - Nom: '{target_col}'")
        print(f"  - Type: {context_df[target_col].dtype}")
        print(f"  - Valeurs non-nulles: {context_df[target_col].notna().sum()} / {len(context_df)}")
        print(f"  - Valeurs manquantes: {context_df[target_col].isna().sum()}")
        print(f"  - Min: {context_df[target_col].min():.4f}")
        print(f"  - Max: {context_df[target_col].max():.4f}")
        print(f"  - Moyenne: {context_df[target_col].mean():.4f}")
    print()
    
    print("📊 Aperçu des données (premières 10 lignes):")
    print(context_df.head(10))
    print()
    
    print("📊 Aperçu des données (dernières 10 lignes):")
    print(context_df.tail(10))
    print()
    
    # Identifier les covariables (colonnes autres que id, timestamp, target)
    covariate_cols = [col for col in context_df.columns 
                     if col not in ['id', 'timestamp', 'target']]
    
    if covariate_cols:
        print(f"📈 Covariables détectées ({len(covariate_cols)}):")
        for col in covariate_cols:
            non_null = context_df[col].notna().sum()
            print(f"  - '{col}': {non_null}/{len(context_df)} valeurs non-nulles")
        print()
    else:
        print("📈 Covariables: Aucune (prédiction univariée pure)\n")
    
    print("="*70)
    print("\n💡 RÉSUMÉ: Ce qui est donné au modèle Chronos-2:")
    print(f"  1. ✅ {len(context_df)} observations historiques")
    print(f"  2. ✅ Colonne 'id': identifie la série temporelle")
    print(f"  3. ✅ Colonne 'timestamp': dates régulières")
    print(f"  4. ✅ Colonne 'target': valeurs historiques à prédire")
    if covariate_cols:
        print(f"  5. ✅ {len(covariate_cols)} covariables: {', '.join(covariate_cols[:3])}{'...' if len(covariate_cols) > 3 else ''}")
    
    # Calculer la longueur de prédiction
    pred_len = min(1024, max(12, len(context_df) // 10))
    print(f"\n   Le modèle va utiliser ces données historiques pour prédire")
    print(f"   les {pred_len} prochaines valeurs.\n")
    
else:
    print("⚠️  Le DataFrame 'context_df' n'existe pas encore.")
    print("   Exécutez d'abord la cellule 'Exemple 1' pour préparer les données.\n")


In [ ]:
# Visualiser les données d'entrée du modèle
import matplotlib.pyplot as plt

print("=== VISUALISATION DES DONNÉES D'ENTRÉE ===\n")

if 'context_df' in globals():
    try:
        fig, ax = plt.subplots(figsize=(14, 6))
        
        # Tracer la série temporelle historique
        ax.plot(context_df['timestamp'], context_df['target'], 
                linewidth=2, color='#2E86AB', marker='o', markersize=3, 
                label='Données historiques (entrée du modèle)')
        
        # Marquer le dernier point (point de départ pour les prédictions)
        last_idx = len(context_df) - 1
        ax.plot(context_df['timestamp'].iloc[last_idx], context_df['target'].iloc[last_idx],
                marker='*', markersize=15, color='red', 
                label='Dernier point (début des prédictions)')
        
        ax.set_xlabel('Date', fontsize=12, fontweight='bold')
        ax.set_ylabel('Valeur (target)', fontsize=12, fontweight='bold')
        ax.set_title('Données historiques fournies au modèle Chronos-2', 
                     fontsize=14, fontweight='bold')
        ax.legend(loc='best', fontsize=10)
        ax.grid(True, alpha=0.3, linestyle='--')
        
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        print("✓ Graphique des données d'entrée créé")
        print(f"\n📊 Statistiques de la série:")
        print(f"   - Nombre de points: {len(context_df)}")
        print(f"   - Période: {context_df['timestamp'].min().strftime('%Y-%m-%d')} à {context_df['timestamp'].max().strftime('%Y-%m-%d')}")
        print(f"   - Valeur min: {context_df['target'].min():.4f}")
        print(f"   - Valeur max: {context_df['target'].max():.4f}")
        print(f"   - Valeur moyenne: {context_df['target'].mean():.4f}")
        print(f"   - Écart-type: {context_df['target'].std():.4f}\n")
        
    except Exception as e:
        print(f"❌ Erreur lors de la visualisation: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  Le DataFrame 'context_df' n'existe pas encore.")
    print("   Exécutez d'abord la cellule 'Exemple 1' pour préparer les données.\n")


In [ ]:
# Exemple 2: Faire des prédictions avec votre dataset
print("=== EXEMPLE 2: INFÉRENCE AVEC VOTRE DATASET ===\n")

# S'assurer que le pipeline est chargé
if 'pipeline' not in globals():
    print("Chargement du pipeline Chronos-2...")
    from chronos import Chronos2Pipeline
    pipeline = Chronos2Pipeline.from_pretrained(
        "amazon/chronos-2",
        device_map="mps"  # ou "cuda" pour GPU NVIDIA, "cpu" pour CPU
    )
    print("✓ Pipeline chargé\n")

# Faire des prédictions
try:
    # Déterminer la longueur de prédiction (par exemple, 10% de la longueur du contexte)
    # Limiter à 1024 car Chronos-2 recommande prediction_length <= 1024
    prediction_length = min(1024, max(12, len(context_df) // 10))
    print(f"Longueur de prédiction: {prediction_length} pas de temps")
    if len(context_df) // 10 > 1024:
        print(f"⚠️  Note: La longueur calculée ({len(context_df) // 10}) a été limitée à 1024 (recommandation Chronos-2)\n")
    else:
        print()
    
    predictions = pipeline.predict_df(
        context_df,
        prediction_length=prediction_length,
        quantile_levels=[0.05, 0.25, 0.5, 0.75, 0.95],  # Quantiles pour intervalles de confiance
        id_column="id",
        timestamp_column="timestamp",
        target="target"
    )
    
    print(f"✓ Prédictions générées avec succès!")
    print(f"  - Shape: {predictions.shape}")
    print(f"  - Colonnes: {predictions.columns.tolist()}\n")
    
    print("Aperçu des prédictions:")
    print(predictions.head(10))
    
except Exception as e:
    print(f"❌ Erreur lors de l'inférence: {e}")
    print(f"   Type: {type(e).__name__}")
    import traceback
    traceback.print_exc()


In [ ]:
# Visualiser les prédictions avec les données historiques
import matplotlib.pyplot as plt

print("=== VISUALISATION DES PRÉDICTIONS ===\n")

try:
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # Filtrer les données pour la série spécifique
    series_id = context_df['id'].iloc[0]
    hist_data = context_df[context_df['id'] == series_id].copy()
    pred_data = predictions[predictions['id'] == series_id].copy()
    
    # Afficher les données historiques
    ax.plot(hist_data['timestamp'], hist_data['target'], 
            label='Données historiques', linewidth=2.5, color='#2E86AB', marker='o', markersize=3)
    
    # Afficher la médiane (quantile 0.5)
    if '0.5' in pred_data.columns:
        ax.plot(pred_data['timestamp'], pred_data['0.5'], 
                label='Prédiction (médiane)', linewidth=2.5, color='#A23B72', 
                linestyle='--', marker='s', markersize=4)
    
    # Afficher les intervalles de confiance
    if '0.05' in pred_data.columns and '0.95' in pred_data.columns:
        ax.fill_between(pred_data['timestamp'], 
                         pred_data['0.05'], 
                         pred_data['0.95'],
                         alpha=0.2, color='red', label='IC 90%')
    
    if '0.25' in pred_data.columns and '0.75' in pred_data.columns:
        ax.fill_between(pred_data['timestamp'], 
                         pred_data['0.25'], 
                         pred_data['0.75'],
                         alpha=0.3, color='red', label='IC 50%')
    
    # Ligne verticale pour séparer historique et prédictions
    if len(hist_data) > 0:
        ax.axvline(x=hist_data['timestamp'].iloc[-1], color='gray', 
                   linestyle=':', linewidth=2, label='Début des prédictions')
    
    ax.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax.set_ylabel(f'Valeur ({target_column})', fontsize=12, fontweight='bold')
    ax.set_title(f'Prévisions Chronos-2 pour {target_column}', fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print("✓ Visualisation créée")
    
except Exception as e:
    print(f"❌ Erreur lors de la visualisation: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Exemple 3: Prédire plusieurs colonnes simultanément (multivarié)
print("=== EXEMPLE 3: PRÉDICTION MULTIVARIÉE ===\n")

# Choisir plusieurs colonnes à prédire
# Exemple: prédire gdp, investment, et production_kg simultanément
target_columns_multivariate = ['gdp', 'investment', 'production_kg']  # Changez selon vos besoins

# Filtrer pour ne garder que les colonnes qui existent
target_columns_multivariate = [col for col in target_columns_multivariate if col in df.columns]

if len(target_columns_multivariate) > 0:
    print(f"Colonnes sélectionnées pour prédiction multivariée: {target_columns_multivariate}\n")
    
    try:
        # Préparer les données pour chaque colonne
        all_context_dfs = []
        
        for target_col in target_columns_multivariate:
            context_df_multi, _, _ = prepare_data_for_chronos2(
                df=df,
                date_column='date',
                target_columns=target_col,
                id_value=f'ecb_{target_col}',
                freq=None
            )
            all_context_dfs.append(context_df_multi)
        
        # Concaténer tous les DataFrames
        context_df_multi = pd.concat(all_context_dfs, ignore_index=True)
        
        print(f"✓ Dataset multivarié préparé:")
        print(f"  - Nombre de séries: {len(target_columns_multivariate)}")
        print(f"  - Nombre total de lignes: {len(context_df_multi)}")
        print(f"  - IDs uniques: {context_df_multi['id'].unique().tolist()}\n")
        
        # Faire des prédictions pour toutes les séries
        # Limiter à 1024 car Chronos-2 recommande prediction_length <= 1024
        prediction_length_multi = min(1024, max(12, len(context_df_multi) // (10 * len(target_columns_multivariate))))
        
        predictions_multi = pipeline.predict_df(
            context_df_multi,
            prediction_length=prediction_length_multi,
            quantile_levels=[0.1, 0.5, 0.9],
            id_column="id",
            timestamp_column="timestamp",
            target="target"
        )
        
        print(f"✓ Prédictions multivariées générées!")
        print(f"  - Shape: {predictions_multi.shape}")
        print(f"\nAperçu:")
        print(predictions_multi.head(15))
        
    except Exception as e:
        print(f"❌ Erreur: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  Aucune colonne valide sélectionnée pour la prédiction multivariée")


In [ ]:
# Exemple 4: Utiliser des covariables pour améliorer les prédictions
print("=== EXEMPLE 4: PRÉDICTION AVEC COVARIABLES ===\n")

# Dans cet exemple, on prédit une colonne cible en utilisant d'autres colonnes comme covariables
target_with_covariates = 'gdp'  # Colonne à prédire

# Choisir des covariables (colonnes qui peuvent aider à prédire la cible)
# Exemple: utiliser investment, production_kg, etc. comme covariables
potential_covariates = ['investment', 'production_kg', 'capacityutil_kg', 'confidence_kg']
covariates_to_use = [col for col in potential_covariates if col in df.columns and col != target_with_covariates]

print(f"Colonne cible: {target_with_covariates}")
print(f"Covariables sélectionnées: {covariates_to_use}\n")

try:
    # Préparer les données avec covariables
    context_df_cov, _, _ = prepare_data_for_chronos2(
        df=df,
        date_column='date',
        target_columns=target_with_covariates,
        covariate_columns=covariates_to_use,
        id_value='ecb_with_covariates',
        freq=None
    )
    
    print(f"✓ Dataset avec covariables préparé:")
    print(f"  - Colonnes: {context_df_cov.columns.tolist()}")
    print(f"  - Nombre de lignes: {len(context_df_cov)}\n")
    
    # Afficher un aperçu
    print("Aperçu des données avec covariables:")
    print(context_df_cov.head(10))
    
    # Note: Pour utiliser les covariables dans les prédictions futures,
    # vous devrez fournir un future_df avec les valeurs futures des covariables
    # Pour l'instant, Chronos-2 utilisera les valeurs passées des covariables
    
    # Limiter à 1024 car Chronos-2 recommande prediction_length <= 1024
    prediction_length_cov = min(1024, max(12, len(context_df_cov) // 10))
    
    predictions_cov = pipeline.predict_df(
        context_df_cov,
        prediction_length=prediction_length_cov,
        quantile_levels=[0.1, 0.5, 0.9],
        id_column="id",
        timestamp_column="timestamp",
        target="target"
    )
    
    print(f"\n✓ Prédictions avec covariables générées!")
    print(f"  - Shape: {predictions_cov.shape}")
    print(f"\nAperçu des prédictions:")
    print(predictions_cov.head(10))
    
except Exception as e:
    print(f"❌ Erreur: {e}")
    import traceback
    traceback.print_exc()

### 📝 Notes importantes pour votre dataset

1. **Format des données**: 
   - La colonne `date` doit être convertible en datetime
   - Les colonnes numériques ne doivent pas avoir trop de valeurs manquantes

2. **Choix de la colonne cible**:
   - Remplacez `'gdp'` dans les exemples par la colonne que vous souhaitez prédire
   - Vous pouvez prédire plusieurs colonnes en les listant dans `target_columns`

3. **Covariables**:
   - Les autres colonnes numériques peuvent être utilisées comme covariables
   - Chronos-2 utilisera automatiquement les valeurs passées des covariables

4. **Longueur de prédiction**:
   - Ajustez `prediction_length` selon votre horizon de prévision souhaité
   - Généralement: 10-20% de la longueur du contexte est raisonnable

5. **Valeurs manquantes**:
   - Chronos-2 peut gérer certaines valeurs manquantes, mais il est recommandé
     de les traiter avant l'inférence (interpolation, forward fill, etc.)
